**Проект NLP**

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

## Подготовка

In [ ]:
# Загрузим библиотеки
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Предобработка датасета

In [ ]:
# Проведем предобработку
display(df[df['text'].isna()])
df[df.duplicated(subset=df.columns.to_list())].sort_values(by='text', ascending=False)
# Все чисто!

,text,toxic


,text,toxic


In [ ]:
# Анализ датасета
df.describe()
# негативных твитов всего 10%

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


**Вывод**
- Пропусков и дублей - нет
- Данные несбалансированы - 10% - негативные твиты.

### Чистка текста

In [ ]:
# Загрузим лемматизатор и проверим его
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("bats"))

bat


In [ ]:
corpus = list(df['text'])

In [ ]:
# Создали функцию для определения части речи
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.VERB)

In [ ]:
# Обработка текста: приведение к стррочным буквам, выделения лемм, удаление стоп-слов, регулярных выражений
stop_words = set(stopwords.words('english'))
def lemmatize(text):
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    without_stop_words = ' '.join([w for w in nltk.word_tokenize(lemmatized_output) if not w in stop_words])
    text = re.sub(r'[^a-zA-Z]', ' ', without_stop_words)
    return " ".join(text.split())
# Проверка
lemmatize(corpus[2])

'hey man m really try edit war s guy constantly remove relevant information talk edits instead talk page seem care format actual info'

**Выводы по обработке текста**
- В тексте много ошибок, особенно в инвективной лексике. Сначала посылают на хер (с ошибкой в слове fcuk, fuckfuckfuck), а потом желают удачного дня - возможно такой твит ML запишет в положительный твит;

In [ ]:
# Обработаем весь датасет
df['lemm_text'] = df['text'].apply(lemmatize)

In [ ]:
# Проверим результат
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,d aww match background colour m seemingly stuc...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m really try edit war s guy constantly...
3,"""\nMore\nI can't make any real suggestions on ...",0,ca n t make real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page s


**Вывод**
- Стоп-слов нет, но остались буквы;
- Глаголы в инфинитиве - все ок.

## Обучение

In [ ]:
# Разделим данные на трейн и тест (75, 25)
X = df.drop(['text', 'toxic'], axis=1)
y = df['toxic']
X_tr, X_t, y_tr, y_t = train_test_split(X, y, stratify=y, test_size=0.25)
corpus_tr = X_tr['lemm_text'].values.astype('U')
corpus_t = X_t['lemm_text'].values.astype('U')

In [ ]:
# Подготовим мешок слов и TF-IDF
count_vect = CountVectorizer()
count_tf_idf = TfidfVectorizer()

In [ ]:
X_tr = count_tf_idf.fit_transform(corpus_tr)
X_t = count_tf_idf.transform(corpus_t)
print("Размер матрицы:", X_tr.shape)
print("Размер матрицы:", X_t.shape)

Размер матрицы: (119678, 132432)
Размер матрицы: (39893, 132432)


In [ ]:
X_tr_c = count_vect.fit_transform(corpus_tr)
X_t_c = count_vect.transform(corpus_t)
print("Размер матрицы:", X_tr_c.shape)
print("Размер матрицы:", X_t_c.shape)

Размер матрицы: (119678, 132432)
Размер матрицы: (39893, 132432)


### Логистическая регрессия

In [ ]:
# Создадим функцю для логистической регрессии
def lr(features_tr, target_tr, features_val, target_val, C, cl_w):
    lr = LogisticRegression(class_weight = cl_w, C=C, solver = 'liblinear', random_state=42)
    lr.fit(features_tr, target_tr)
    predictions = lr.predict(features_val)
    precision = precision_score(target_val, predictions)
    recall = recall_score(target_val, predictions)
    f1 = f1_score(target_val, predictions)
    print("F1 = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        f1, precision, recall))
    print(confusion_matrix(target_val, predictions))

In [ ]:
lr(X_tr, y_tr, X_t, y_t, 10, None)

F1 = 0.78 | Точность = 0.879, Полнота = 0.704
[[35442   395]
 [ 1199  2857]]


In [ ]:
lr(X_tr_c, y_tr, X_t_c, y_t, 10, None)

F1 = 0.70 | Точность = 0.895, Полнота = 0.571
[[35566   271]
 [ 1741  2315]]


/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### LinearSVC

In [ ]:
# Создадим функцю для LinearSVC
def lsvc(features_tr, target_tr, features_val, target_val, C, cl_w):
    lsvc = LinearSVC(class_weight = cl_w, C=C, random_state=42)
    lsvc.fit(features_tr, target_tr)
    predictions = lsvc.predict(features_val)
    precision = precision_score(target_val, predictions)
    recall = recall_score(target_val, predictions)
    f1 = f1_score(target_val, predictions)
    print("F1 = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        f1, precision, recall))
    print(confusion_matrix(target_val, predictions))

In [ ]:
lsvc(X_tr, y_tr, X_t, y_t, 0.8, None)

F1 = 0.79 | Точность = 0.885, Полнота = 0.707
[[35463   374]
 [ 1190  2866]]


In [ ]:
lsvc(X_tr_c, y_tr, X_t_c, y_t, 1, None)

F1 = 0.75 | Точность = 0.779, Полнота = 0.732
[[34995   842]
 [ 1087  2969]]


### RandomForest

In [ ]:
# Создадим функцю для forest
def forest(features_tr, target_tr, features_val, target_val):
    forest = RandomForestClassifier()
    forest.fit(features_tr, target_tr)
    predictions = forest.predict(features_val)
    precision = precision_score(target_val, predictions)
    recall = recall_score(target_val, predictions)
    f1 = f1_score(target_val, predictions)
    print("F1 = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        f1, precision, recall))
    print(confusion_matrix(target_val, predictions))

In [ ]:
forest(X_tr, y_tr, X_t, y_t)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


F1 = 0.65 | Точность = 0.913, Полнота = 0.508
[[35641   196]
 [ 1996  2060]]


In [ ]:
forest(X_tr_c, y_tr, X_t_c, y_t)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


F1 = 0.65 | Точность = 0.889, Полнота = 0.509
[[35580   257]
 [ 1993  2063]]


### LGBM

In [ ]:
# Создадим функцю для LGBMClassifier
def lgbm(features_tr, target_tr, features_val, target_val, lr):
    lgbm = LGBMClassifier(learning_rate=lr, random_state=42)
    lgbm.fit(features_tr, target_tr, eval_set=[(features_val, target_val)], eval_metric='F1')
    predictions = lgbm.predict(features_val)
    precision = precision_score(target_val, predictions)
    recall = recall_score(target_val, predictions)
    f1 = f1_score(target_val, predictions)
    print("F1 = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        f1, precision, recall))
    print(confusion_matrix(target_val, predictions))

In [ ]:
#lgbm(X_tr, y_tr, X_t, y_t, 0.25)

## Выводы

In [ ]:
# Сравним результаты тестирования моделей
data1 = {'model':['log_reg_tf', 'log_reg_bag', 'linearSVC_tf', 'linearSVC_bag', 'rand_forest_tf', 'rand_forest_bag', 'lgbm_tf'], 'F1':[0.78, 0.70, 0.79, 0.75, 0.65, 0.65, 0.78]}
table = pd.DataFrame(data1)
table.sort_values(by='F1',ascending=False)

,model,F1
2,linearSVC_tf,0.79
0,log_reg_tf,0.78
6,lgbm_tf,0.78
3,linearSVC_bag,0.75
1,log_reg_bag,0.70
4,rand_forest_tf,0.65
5,rand_forest_bag,0.65


**Вывод:**
- Лучшая модель - LinearSVC - 0.79 (С - 0.8), меньше всего ошибок FP и FN;
- Модели логист.регрессия и LGBMClassifier дали близкие результаты, но с разницей в точности;
- TF-IDF дает лучшие результаты, чем bag_of_words;
- В твитах много ошибок - сначала ругательство с ошибкой, а потом Nice day, скорее всего ML определит как положительный твит:)
- Разбивка на биграммы и триграммы - ухудшает модель;